<a href="https://colab.research.google.com/github/girisiman/Deep_Learning_Tasks/blob/master/GAN_1_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
bs = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transform, download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)


  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27476936.06it/s]                            


Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 465822.12it/s]
  1%|          | 16384/1648877 [00:00<00:11, 147819.74it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7232656.74it/s]                           
8192it [00:00, 185282.64it/s]


Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [3]:
train_dataset


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist_data/
    Split: Train

In [0]:
batch_size = 128
n_colours = 1
z_size = 100
G_h_size = 28
D_h_size = 28
image_size = 28

In [0]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable


import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transf
import torchvision.models as models
import torchvision.utils as vutils
import torch.nn.functional as F


In [0]:
image_size = 28
G_h_size = 28
class DCGAN_G(torch.nn.Module):
	def __init__(self):
    
    #self.image_size = image_size
    #self.G_h_size = G_h_size
    #self.z_size = z_size
    
		super(DCGAN_G, self).__init__()
		main = torch.nn.Sequential()

		# We need to know how many layers we will use at the beginning
		mult = image_size // 8

		### Start block
		# Z_size random numbers
		main.add_module('Start-ConvTranspose2d', torch.nn.ConvTranspose2d(z_size, G_h_size * mult, kernel_size=2, stride=1, padding=0, bias=False))
		if F.selu:
			main.add_module('Start-SELU', torch.nn.SELU(inplace=True))
		else:
			main.add_module('Start-BatchNorm2d', torch.nn.BatchNorm2d(G_h_size * mult))
			main.add_module('Start-ReLU', torch.nn.ReLU())
		# Size = (G_h_size * mult) x 4 x 4

		### Middle block (Done until we reach ? x image_size/2 x image_size/2)
		i = 1
		while mult > 1:
			main.add_module('Middle-ConvTranspose2d [%d]' % i, torch.nn.ConvTranspose2d(G_h_size * mult, G_h_size * (mult//2), kernel_size=2, stride=1, padding=1, bias=False))
			if torch.nn.SELU:
				main.add_module('Middle-SELU [%d]' % i, torch.nn.SELU(inplace=True))
			else:
				main.add_module('Middle-BatchNorm2d [%d]' % i, torch.nn.BatchNorm2d(G_h_size * (mult//2)))
				main.add_module('Middle-ReLU [%d]' % i, torch.nn.ReLU())
			# Size = (G_h_size * (mult/(2*i))) x 8 x 8
			mult = mult // 2
			i += 1

		### End block
		# Size = G_h_size x image_size/2 x image_size/2
		main.add_module('End-ConvTranspose2d', torch.nn.ConvTranspose2d(G_h_size, 1, kernel_size=2, stride=1, padding=1, bias=False))
		main.add_module('End-Tanh', torch.nn.Tanh())
		# Size = n_colors x image_size x image_size
		self.main = main

	def forward(self, input):
		output = self.main(input)
		return output

In [0]:
z_size = 100
g = DCGAN_G()

In [116]:
g

DCGAN_G(
  (main): Sequential(
    (Start-ConvTranspose2d): ConvTranspose2d(100, 84, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (Start-SELU): SELU(inplace)
    (Middle-ConvTranspose2d [1]): ConvTranspose2d(84, 28, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Middle-SELU [1]): SELU(inplace)
    (End-ConvTranspose2d): ConvTranspose2d(28, 1, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (End-Tanh): Tanh()
  )
)

In [0]:
D_h_size = 28
class DCGAN_D(torch.nn.Module):
	def __init__(self):
		super(DCGAN_D, self).__init__()
		main = torch.nn.Sequential()

		### Start block
		# Size = n_colors x image_size x image_size
		main.add_module('Start-Conv2d', torch.nn.Conv2d(1, D_h_size, kernel_size=2, stride=1, padding=1, bias=False))
		if torch.nn.SELU:
			main.add_module('Start-SELU', torch.nn.SELU(inplace=True))
		else:
			main.add_module('Start-LeakyReLU', torch.nn.LeakyReLU(0.2, inplace=True))
		image_size_new = image_size // 2
		# Size = D_h_size x image_size/2 x image_size/2

		### Middle block (Done until we reach ? x 4 x 4)
		mult = 1
		i = 0
		while image_size_new > 4:
			main.add_module('Middle-Conv2d [%d]' % i, torch.nn.Conv2d(D_h_size * mult, D_h_size * (2*mult), kernel_size=2, stride=1, padding=1, bias=False))
			if torch.nn.SELU:
				main.add_module('Middle-SELU [%d]' % i, torch.nn.SELU(inplace=True))
			else:
				main.add_module('Middle-BatchNorm2d [%d]' % i, torch.nn.BatchNorm2d(D_h_size * (2*mult)))
				main.add_module('Middle-LeakyReLU [%d]' % i, torch.nn.LeakyReLU(0.2, inplace=True))
			# Size = (D_h_size*(2*i)) x image_size/(2*i) x image_size/(2*i)
			image_size_new = image_size_new // 2
			mult *= 2
			i += 1

		### End block
		# Size = (D_h_size * mult) x 4 x 4
		main.add_module('End-Conv2d', torch.nn.Conv2d(D_h_size * mult, 1, kernel_size=2, stride=1, padding=0, bias=False))
		main.add_module('End-Sigmoid', torch.nn.Sigmoid())
		# Size = 1 x 1 x 1 (Is a real cat or not?)
		self.main = main

	def forward(self, input):
		if isinstance(input.data, torch.cuda.FloatTensor) and param.n_gpu > 1:
			output = torch.nn.parallel.data_parallel(self.main, input, range(param.n_gpu))
		else:
			output = self.main(input)
		# Convert from 1 x 1 x 1 to 1 so that we can compare to given label (cat or not?)
		return output.view(-1)


In [0]:
d = DCGAN_D()

In [119]:
d

DCGAN_D(
  (main): Sequential(
    (Start-Conv2d): Conv2d(1, 28, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Start-SELU): SELU(inplace)
    (Middle-Conv2d [0]): Conv2d(28, 56, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Middle-SELU [0]): SELU(inplace)
    (Middle-Conv2d [1]): Conv2d(56, 112, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Middle-SELU [1]): SELU(inplace)
    (End-Conv2d): Conv2d(112, 1, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (End-Sigmoid): Sigmoid()
  )
)

In [0]:
def weights_init(m):
	classname = m.__class__.__name__
	if classname.find('Conv') != -1:
		m.weight.data.normal_(0.0, 0.02)
	elif classname.find('BatchNorm') != -1:
		# Estimated variance, must be around 1
		m.weight.data.normal_(1.0, 0.02)
		# Estimated mean, must be around 0
		m.bias.data.fill_(0)

In [121]:
g.apply(weights_init)



DCGAN_G(
  (main): Sequential(
    (Start-ConvTranspose2d): ConvTranspose2d(100, 84, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (Start-SELU): SELU(inplace)
    (Middle-ConvTranspose2d [1]): ConvTranspose2d(84, 28, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Middle-SELU [1]): SELU(inplace)
    (End-ConvTranspose2d): ConvTranspose2d(28, 1, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (End-Tanh): Tanh()
  )
)

In [122]:
d.apply(weights_init)

DCGAN_D(
  (main): Sequential(
    (Start-Conv2d): Conv2d(1, 28, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Start-SELU): SELU(inplace)
    (Middle-Conv2d [0]): Conv2d(28, 56, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Middle-SELU [0]): SELU(inplace)
    (Middle-Conv2d [1]): Conv2d(56, 112, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1), bias=False)
    (Middle-SELU [1]): SELU(inplace)
    (End-Conv2d): Conv2d(112, 1, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (End-Sigmoid): Sigmoid()
  )
)

In [81]:
# Load existing models
if G_load != '':
	g.load_state_dict(torch.load(G_load))
if D_load != '':
	d.load_state_dict(torch.load(D_load))


NameError: ignored

In [0]:
criterion = nn.BCELoss()


In [0]:
mb_size = 64

transform = transforms.ToTensor()

trainData = torchvision.datasets.MNIST('./data/', download=True, transform=transform, train=True)

trainLoader = torch.utils.data.DataLoader(trainData, shuffle=True, batch_size=mb_size)

In [123]:
import random
seed = random.randint(1, 10000)
random.seed(seed)
torch.manual_seed(seed)

In [0]:
batch_size = 28
## Transforming images
trans = transf.Compose([
	transf.Resize((image_size)),
	# This makes it into [0,1]
	transf.ToTensor(),
	# This makes it into [-1,1] so tanh will work properly
	transf.Normalize(mean = [0.5], std = [0.5])
])

## Importing dataset
data = dset.MNIST(root='./mnist_data/', train=True, transform=trans, download=True)

# Loading data in batch
dataset = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

In [127]:
data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist_data/
    Split: Train

In [0]:
# Based on DCGAN paper, they found using betas[0]=.50 better.
# betas[0] represent is the weight given to the previous mean of the gradient
# betas[1] is the weight given to the previous variance of the gradient
lr_D = 0.00005
lr_G = 0.0002
beta1 = 0.5
optimizerd = torch.optim.Adam(d.parameters(), lr=lr_D, betas=(beta1, 0.999))
optimizerg = torch.optim.Adam(g.parameters(), lr=lr_G, betas=(beta1, 0.999))

In [131]:

# Soon to be variables
x = torch.FloatTensor(batch_size, n_colours, image_size, image_size)
y = torch.FloatTensor(batch_size)
z = torch.FloatTensor(batch_size, z_size, 1, 1)
# This is to see during training, size and values won't change
z_test = torch.FloatTensor(batch_size, z_size, 1, 1).normal_(0, 1)
# Now Variables
x = Variable(x)
y = Variable(y)
z = Variable(z)
z_test = Variable(z_test)

for epoch in range(20):
  err_d_rum = 0.0
  err_g_run = 0.0
  for i, data_batch in enumerate(dataset,0):
		########################
		# (1) Update D network #
		########################
    for p in d.parameters():
      p.requires_grad = True

		# Train with real data
    d.zero_grad()
		# We can ignore labels since they are all cats!
    images, labels = data_batch
    #print(images)
		# Mostly necessary for the last one because if N might not be a multiple of batch_size
    current_batch_size = images.size(0)
    print(current_batch_size)
		# Transfer batch of images to x
    x.data.resize_as_(images).copy_(images)
    print(x.size())
		# y is now a vector of size current_batch_size filled with 1
    y.data.resize_(current_batch_size).fill_(1)
    print(y.size())
    y_pred = d(images)
    print(y_pred.size())
    errD_real = criterion(y_pred, y)
    errD_real.backward()
		# Var has data and gradient element, we keep the mean of the data element
    D_real = y_pred.data.mean()

		# Train with fake data
    z.data.resize_(current_batch_size, z_size, 1, 1).normal_(0, 1)
    x_fake = g(z)
    y.data.resize_(current_batch_size).fill_(0)
		# Detach y_pred from the neural network G and put it inside D
    y_pred_fake = d(x_fake.detach())
    errD_fake = criterion(y_pred_fake, y)
    errD_fake.backward()
    D_fake = y_pred_fake.data.mean()
    errD = errD_real + errD_fake
    optimizerd.step()
    # (2) Update G network #
		########################

		# Make it a tiny bit faster
    for p in d.parameters():
      p.requires_grad = False
    g.zero_grad()
		# Generator wants to fool discriminator so it wants to minimize loss of discriminator assuming label is True
    y.data.resize_(current_batch_size).fill_(1)
    y_pred_fake = d(x_fake)
    errG = criterion(y_pred_fake, y)
    errG.backward(retain_graph=True)
    D_G = y_pred_fake.data.mean()
    optimizerg.step()
    current_step = i + epoch*len(dataset)
    err_d_run += err_d.item()
    err_g_run += err_g.item()
print('Epoch:{},   G_loss:{},    D_loss:{}'.format(epoch, err_g_run/(i+1), err_d_run/(i+1)))
samples = G(z).detach()
samples = samples.view(samples.size(0), 1, 28, 28).cpu()
imshow(samples)
    

28
torch.Size([28, 1, 28, 28])
torch.Size([28])
torch.Size([25200])


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([28])) that is different to the input size (torch.Size([25200])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


ValueError: ignored

In [0]:
x = torch.FloatTensor(batch_size, n_colours, image_size, image_size)

In [35]:
x

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,         nan,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.1491e-43,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00, -3.3125e+26,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00, -9.4780e+26,  1.3566e-21,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          ...,
   

In [0]:
y_ = d(x)

In [99]:
y_


tensor([0.4985, 0.4980, 0.4947,  ..., 0.5012, 0.5003, 0.5027],
       grad_fn=<ViewBackward>)

In [92]:
y

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])

In [102]:
y_.size()

torch.Size([115200])

In [103]:
y.size()

torch.Size([128])